In [1]:
import os
import uproot as ur
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bitstring import BitArray

In [2]:
paths = []

for (path, dirnames, filenames) in os.walk('/mnt/scratch3/dmisra/zdcdata_current/'):
    paths.extend(os.path.join(path, name) for name in filenames)

In [3]:
samples = {}

for path in paths:
    with ur.open(path) as file:
       tree = file["events"]
       samples[os.path.basename(f'{path}')] = tree.arrays()

Bit Manipulation

In [4]:
def bitExtract(pattern, length, position):  
    return ((1 << length) - 1)  &  (pattern >> (position - 1))

In [5]:
def signedint(xbits):
    x_int = []
    x_bin = np.vectorize(np.binary_repr, otypes=[str])(xbits, width=12)
    for bits in x_bin:
        if bits[0] == 0:
             x_int.append(BitArray(bin=bits).int)
        else:
            x_int.append(-BitArray(bin=bits[1:]).int)
    return np.array(x_int)

Energy Deposition per Cell

In [6]:
branches = ['ZDC_SiliconPix_Hits', 'ZDC_WSi_Hits', 'ZDC_PbSi_Hits', 'ZDCHcalHits']

In [7]:
def cell_features(data, count, branch):
    event_features = []
    energy_labels = []
    
    for i in range(count):
        label = np.sqrt(data["MCParticles.momentum.x"][0,0]**2 + data["MCParticles.momentum.y"][0,0]**2 + data["MCParticles.momentum.z"][0,0]**2)
        energy_labels.append(label)
        
        energies = np.array(data[f"{branch}.energy"][i])
        cellID = np.array(data[f"{branch}.cellID"][i])
        layerID = bitExtract(cellID, 6, 9)
        idx = signedint(bitExtract(cellID, 11, 25))
        idy = signedint(bitExtract(cellID, 11, 37))
        
        df = pd.DataFrame((zip(idx, idy, layerID, energies)), columns=['idx', 'idy', 'layerID', 'energy'])
        df_grouped = df.groupby(["layerID", "idx", "idy"])['energy'].sum().reset_index().replace(np.NaN, 0.)

        event_features.append(np.array(df_grouped))
        
    return event_features, energy_labels

Graph Convolutional Network

In [8]:
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data, Dataset, DataLoader
from torch_geometric.nn import knn_graph, GCNConv, global_add_pool

/home/dmisra/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device
torch.manual_seed(42)

In [10]:
merged_data = [np.concatenate(list(samples.values()))]

In [11]:
data_features = cell_features(merged_data[0], 40000, branches[1])[0]

In [12]:
data_labels = cell_features(merged_data[0], 40000, branches[1])[1]
np.savetxt('/home/dmisra/eic/zdc_data/gnn_labels.csv', data_labels)

In [ ]:
data_labels = np.loadtxt('/home/dmisra/eic/zdc_data/gnn_labels.csv', delimiter=',')

In [14]:
labels = torch.from_numpy(np.array(data_labels))

In [15]:
def graph_set(data):
    graph_set = []
    for i in range(40000):
        tensor = torch.from_numpy(data[i].astype(float))
        graph = knn_graph(tensor[:, [0,1,2]], 8)
        graph_set.append(graph)

    return graph_set

In [16]:
graphs = graph_set(data_features)

: 

: 

In [ ]:
#Split train/test data
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, num_node_features=4):
        super(Net, self).__init__()
        
        #(3 -> 32)
        self.conv1 = GCNConv(num_node_features, 32)
        
        #(32 -> 1)
        self.output = torch.nn.Linear(32, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        #add a batch index, in in case we are running on a single graph
        if not hasattr(data, "batch"):
            data.batch = torch.zeros(len(x), dtype=torch.int64).to(x.device)
        
        #Transform the nodes with the graph convolution
        transformed_nodes = self.conv1(x, edge_index)
        transformed_nodes = torch.nn.functional.elu(transformed_nodes)
        
        #Sum up all the node vectors in each graph according to the batch index
        per_graph_aggregation = global_add_pool(transformed_nodes, data.batch)
        
        #For each graph,
        #predict the classification output based on the total vector
        #from the previous aggregation step
        output = self.output(per_graph_aggregation)
        return output

In [ ]:
net = Net()

In [ ]:
learning_rate = 0.0001
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model_2.parameters(),lr=learning_rate)

In [ ]:
#Set number of epochs
epochs = 50000

#Create lists to track loss values
train_loss_values = []
test_loss_values = []
epoch_count = []

for epoch in range(epochs):
    ###Training
    net.train()
    y_pred = net(x_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    ###Testing
    net.eval()

    with torch.inference_mode():
        test_pred = net(x_test)
        test_loss = loss_fn(test_pred, y_test.type(torch.float))

        if epoch % 100 == 0:
            epoch_count.append(epoch)
            train_loss_values.append(loss.detach().numpy())
            test_loss_values.append(test_loss.detach().numpy())
            print(f"Epoch: {epoch} | MSE Train Loss: {loss} | MSE Test Loss: {test_loss}")


In [ ]:
# Plot the loss curves
plt.plot(epoch_count, train_loss_values, label="Train loss")
plt.plot(epoch_count, test_loss_values, label="Test loss")
plt.title("Training and Test Loss Curves")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend()

Predictions

In [ ]:
from scipy.stats import norm
from scipy.optimize import curve_fit

In [ ]:
#Set the model in evaluation mode
model_1.eval()

#Setup the inference mode context manager
with torch.inference_mode():
  y_preds = model_1(x_test)

plt.hist(y_preds[:,0].numpy(),100,histtype='step')
plt.xlabel('Energy (GeV)')
plt.ylabel('Count')
plt.title('Predicted Energy Distribution')

In [ ]:
#Set the model in evaluation mode
model_1.eval()

#Setup the inference mode context manager
with torch.inference_mode():
  y_preds_200GeV = model_1(features_200GeV)
  y_preds_100GeV = model_1(features_100GeV)
  y_preds_50GeV = model_1(features_50GeV)
  y_preds_10GeV = model_1(features_10GeV)

In [ ]:
peak_preds = norm.fit(y_preds_10GeV)[0], norm.fit(y_preds_50GeV)[0], norm.fit(y_preds_100GeV)[0], norm.fit(y_preds_200GeV)[0]
true_peaks = [10,50,100,200]
peak_preds

In [ ]:
plt.scatter(true_peaks,peak_preds)
plt.xlabel('Particle Energy (GeV)')
plt.ylabel('Reconstructed Energy (GeV)')
plt.plot(np.arange(1,201),np.arange(1,201))
plt.title('Linearity')

In [ ]:
#Get energy resolution from distribution of predictions
def res(preds,energy):
    return norm.fit(preds)[1]/energy

energy_list = [200,100,50,10]
resolutions = res(y_preds_200GeV,200), res(y_preds_100GeV,100), res(y_preds_50GeV,50), res(y_preds_10GeV,10)

In [ ]:
#Curve fit for energy resolution as a function of energy
def f(E,a):
    return a/np.sqrt(E)

popt, pcov = curve_fit(f, energy_list, resolutions)

In [ ]:
popt, pcov

In [ ]:
plt.plot(range(200),f(range(1,201),popt[0]))
plt.scatter(energy_list,resolutions)
plt.xlabel('Energy (GeV)')
plt.ylabel('Resolution')
plt.title('Energy Resolution')

In [ ]:
torch.save(obj=model_1.state_dict(), f="/home/dmisra/eic/model_1")